# Project 1: Digit Classification with KNN and Naive Bayes

In this project, you'll implement your own image recognition system for classifying digits. Read through the code and the instructions carefully and add your own code where indicated. Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but <b> please prepare your own write-up (with your own code). </b>

If you're interested, check out these links related to digit recognition:

Yann Lecun's MNIST benchmarks: http://yann.lecun.com/exdb/mnist/

Stanford Streetview research and data: http://ufldl.stanford.edu/housenumbers/

In [38]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Import a bunch of libraries.
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_mldata
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report

# Set the randomizer seed so results are the same each time.
np.random.seed(0)

Load the data. Notice that we are splitting the data into training, development, and test. We also have a small subset of the training data called mini_train_data and mini_train_labels that you should use in all the experiments below, unless otherwise noted.

In [39]:
# Load the digit data either from mldata.org, or once downloaded to data_home, from disk. The data is about 53MB so this cell
# should take a while the first time your run it.
mnist = fetch_mldata('MNIST original', data_home='~/datasets/mnist')
X, Y = mnist.data, mnist.target

# Rescale grayscale values to [0,1].
X = X / 255.0

# Shuffle the input: create a random permutation of the integers between 0 and the number of data points and apply this
# permutation to X and Y.
# NOTE: Each time you run this cell, you'll re-shuffle the data, resulting in a different ordering.
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]

print 'data shape: ', X.shape
print 'label shape:', Y.shape

# Set some variables to hold test, dev, and training data.
test_data, test_labels = X[61000:], Y[61000:]
dev_data, dev_labels = X[60000:61000], Y[60000:61000]
train_data, train_labels = X[:60000], Y[:60000]
mini_train_data, mini_train_labels = X[:1000], Y[:1000]

data shape:  (70000, 784)
label shape: (70000,)


In [40]:
print test_labels[0:10]
print type(test_labels)
print test_labels.size
print mini_train_labels.size

[ 6.  0.  9.  3.  5.  1.  3.  3.  0.  7.]
<type 'numpy.ndarray'>
9000
1000


(1) Create a 10x10 grid to visualize 10 examples of each digit. Python hints:

- plt.rc() for setting the colormap, for example to black and white
- plt.subplot() for creating subplots
- plt.imshow() for rendering a matrix
- np.array.reshape() for reshaping a 1D feature vector into a 2D matrix (for rendering)

In [ ]:
def P1(num_examples=10):

### STUDENT START ###
    count_list = [0]*10
    complete = False
    labels = test_labels.astype(int)
    for i in range(test_labels.size):
        if(not complete):
            #print 'i ', i, 'Complete: ', complete
            value = labels[i]
            if(count_list[value]<num_examples ):
                count_list[value] +=1
                #plt.subplots(10,10)
                #plt.imshow(np.ndarray.reshape(test_data[value],[28,28],order='C'))
            complete = count_list[0] == num_examples and count_list[1] == num_examples and count_list[2] == num_examples and count_list[3] == num_examples and count_list[4] == num_examples and count_list[5] == num_examples and count_list[6] == num_examples and count_list[7] == num_examples and count_list[8] == num_examples and count_list[9] == num_examples
        else:
            break
    print count_list        
#     digits = list(range(10))
#     examples = np.empty([10,10])
#     pick_count = num_examples*10
#     index =0
#     print examples.shape
#     while (pick_count>0):
#         value = test_labels[index]
#         print index
#         print value
#         if(np.ndarray.size(examples[value])<num_examples):
#             #print value
#             pick_count-=1
#         index+=1
#     print 'end while loop'
    #plt.subplots(10,10)
    #plt.imshow(np.ndarray.reshape(test_data[0],[28,28],order='C'))
    #plt.plot(np.array.reshape(test_data(0,)))

### STUDENT END ###

P1(10)

(2) Evaluate a K-Nearest-Neighbors model with k = [1,3,5,7,9] using the mini training set. Report accuracy on the dev set. For k=1, show precision, recall, and F1 for each label. Which is the most difficult digit?

- KNeighborsClassifier() for fitting and predicting
- classification_report() for producing precision, recall, F1 results

In [ ]:
def P2(k_values):

### STUDENT START ###
    print mini_train_data.size 
    for i in k_values:
        model = KNeighborsClassifier(n_neighbors=i)
        model.fit(mini_train_data,mini_train_labels)
        predicted_labels = model.predict(dev_data)
        wrong_prediction= (predicted_labels != dev_labels)
        print(classification_report(predicted_labels,dev_labels))

### STUDENT END ###

k_values = [1, 3, 5, 7, 9]
P2(k_values)

ANSWER: 2 and 5 are the two digits with consistently low lowest support. So they would the be the digts most difficult to predict.

(3) Using k=1, report dev set accuracy for the training set sizes below. Also, measure the amount of time needed for prediction with each training size.

- time.time() gives a wall clock value you can use for timing operations

In [ ]:
def P3(train_sizes, accuracies):

### STUDENT START ###
    k = 1
    for i in train_sizes:
        print 'Training Set Size: ',i
        training_subset_data = train_data[0:i]
        training_subset_label = train_labels[0:i]
        model = KNeighborsClassifier(k)
        model.fit(training_subset_data,training_subset_label)
        start = time.time()
        predicted_labels = model.predict(dev_data)
        finish  = time.time()
        wrong_prediction = (predicted_labels != dev_labels)
        accuracies.append(np.divide(np.sum(wrong_prediction,dtype=np.float32),predicted_labels.size))
        print 'Time Taken for prediction with ', i , ' samples is :' ,finish-start
    print 'Accuracies ', accuracies
### STUDENT END ###

train_sizes = [100, 200, 400, 800, 1600, 3200, 6400, 12800, 25000]
accuracies = []
P3(train_sizes, accuracies)

(4) Fit a regression model that predicts accuracy from training size. What does it predict for n=60000? What's wrong with using regression here? Can you apply a transformation that makes the predictions more reasonable?

- Remember that the sklearn fit() functions take an input matrix X and output vector Y. So each input example in X is a vector, even if it contains only a single value.

In [ ]:
def P4():

### STUDENT START ###
        #y = LinearRegression.fit(np.array(train_sizes), accuracies)

### STUDENT END ###

P4()

ANSWER:

Fit a 1-NN and output a confusion matrix for the dev data. Use the confusion matrix to identify the most confused pair of digits, and display a few example mistakes.

- confusion_matrix() produces a confusion matrix

In [ ]:
def P5():

### STUDENT START ###
    model = KNeighborsClassifier(1)
    print ('Starting to train model')
    model.fit(train_data,train_labels)
    print ('Completed training. Starting prediction')
    predicted_labels=model.predict(dev_data)
    print(classification_report(predicted_labels,dev_labels))
    print ('Computing Confusion Matrix')
    #print(confusion_matrix(dev_labels,predicted_labels))
    
### STUDENT END ###

P5()

(6) A common image processing technique is to smooth an image by blurring. The idea is that the value of a particular pixel is estimated as the weighted combination of the original value and the values around it. Typically, the blurring is Gaussian -- that is, the weight of a pixel's influence is determined by a Gaussian function over the distance to the relevant pixel.

Implement a simplified Gaussian blur by just using the 8 neighboring pixels: the smoothed value of a pixel is a weighted combination of the original value and the 8 neighboring values. Try applying your blur filter in 3 ways:
- preprocess the training data but not the dev data
- preprocess the dev data but not the training data
- preprocess both training and dev data

Note that there are Guassian blur filters available, for example in scipy.ndimage.filters. You're welcome to experiment with those, but you are likely to get the best results with the simplified version I described above.

In [ ]:
def P6():
    
### STUDENT START ###
    #for i in range(train_data.size):
        print 'Bluring Data'
        training_data_blur = process_data(train_data)
        dev_data_blur  = process_data(dev_data)
        
        print 'Fitting First Scenario - Blurred Training Data'
        model1 = KNeighborsClassifier(1)
        model1.fit(training_data_blur, train_labels)
        model1_predicted_labels = model1.predict(dev_data)
        print(classification_report(model1_predicted_labels,dev_labels))
        
        
        print 'Fitting Second Scenario - Blurred Dev Data'
        model2 = KNeighborsClassifier(1)
        model2.fit(train_data, train_labels)
        model2_predicted_labels = model2.predict(dev_data_blur)
        print(classification_report(model2_predicted_labels,dev_labels))
        
        
        print 'Fitting Third Scenario - Blurred Training and Dev Data'
        model3 = KNeighborsClassifier(1)
        model3.fit(training_data_blur, train_labels)
        model3_predicted_labels = model3.predict(dev_data_blur)
        print(classification_report(model3_predicted_labels,dev_labels))

def process_data(raw_data):
    formatted_data = np.empty_like(raw_data)
    for i in range(len(raw_data)):
        formatted_data[i]=smooth(raw_data[i])
    return formatted_data
        
def smooth(data):
    data_formatted = []
    for i in range(28):
        for j in range(28):
            if i == 0:
                if j == 0:
                    data_formatted.append(np.divide(np.sum([get_value(i,j,data),get_value(i+1,j,data),get_value(i+1,j+1,data),get_value(i,j+1,data)]),4))
                elif j == 27:
                    data_formatted.append(np.divide(np.sum([get_value(i,j,data),get_value(i,j-1,data),get_value(i+1,j-1,data),get_value(i+1,j,data)]),4))
                else:
                    data_formatted.append(np.divide(np.sum([get_value(i,j,data),get_value(i,j-1,data),get_value(i+1,j-1,data),get_value(i+1,j,data),get_value(i+1,j+1,data),get_value(i,j+1,data)]),6))
            elif i == 27:
                if j == 0:
                    data_formatted.append(np.divide(np.sum([get_value(i,j,data),get_value(i-1,j,data),get_value(i-1,j-1,data),get_value(i,j+1,data)]),4))
                elif j == 27:
                    data_formatted.append(np.divide(np.sum([get_value(i,j,data),get_value(i-1,j,data),get_value(i-1,j-1,data),get_value(i-1,j,data)]),4))
                else:
                    data_formatted.append(np.divide(np.sum([get_value(i,j,data),get_value(i,j-1,data),get_value(i-1,j-1,data),get_value(i-1,j,data),get_value(i-1,j+1,data),get_value(i,j+1,data)]),6))               
            else:
                if j == 0:
                    data_formatted.append(np.divide(np.sum([get_value(i,j,data),get_value(i+1,j,data),get_value(i+1,j+1,data),get_value(i,j+1,data),get_value(i-1,j+1,data),get_value(i-1,j,data)]),6))
                elif j == 27:
                    data_formatted.append(np.divide(np.sum([get_value(i,j,data),get_value(i,j-1,data),get_value(i+1,j-1,data),get_value(i+1,j,data),get_value(i-1,j,data),get_value(i-1,j-1,data)]),6))
                else:
                    data_formatted.append(np.divide(np.sum([get_value(i,j,data),get_value(i,j-1,data),get_value(i+1,j-1,data),get_value(i+1,j,data),get_value(i+1,j+1,data),get_value(i,j+1,data),get_value(i-1,j+1,data),get_value(i-1,j,data),get_value(i-1,j-1,data)]),9))
    return data_formatted
#This method computes what position in a vector corresponds to a 2d matrix
def get_value(row,column,data):
    max_row, max_column = 28, 28
    return data[((row % max_row) + (column % max_column)) + (row * (max_column-1))]
        
### STUDENT END ###

P6()

ANSWER:

(7) Fit a Naive Bayes classifier and report accuracy on the dev data. Remember that Naive Bayes estimates P(feature|label). While sklearn can handle real-valued features, let's start by mapping the pixel values to either 0 or 1. You can do this as a preprocessing step, or with the binarize argument. With binary-valued features, you can use BernoulliNB. Next try mapping the pixel values to 0, 1, or 2, representing white, grey, or black. This mapping requires MultinomialNB. Does the multi-class version improve the results? Why or why not?

In [52]:
def P7():

### STUDENT START ###
    
    # Binarizing Data
    print "Converting Data to Binomial"
    b_train_data = binarize_data(train_data,[0.5])
    b_test_data = binarize_data(test_data,[0.5])
    
    #Bernoulli Classifier
    bernoulli_classifier = BernoulliNB(1)
    bernoulli_model = bernoulli_classifier.fit(b_train_data,train_labels)
    b_prediction = bernoulli_model.score(b_test_data,test_labels )
    print 'Prediction with Binomial', b_prediction
    
    #Converting to Multinomial Variable
    print "Converting Data to Multinomial"
    m_traning_data = multinomialize_data(train_data)
    m_test_data = multinomialize_data(test_data)
    multinomial_classifier = MultinomialNB(1)
    multinomial_data_model = multinomial_classifier.fit(m_traning_data,train_labels)
    m_prediction = multinomial_data_model.score(m_test_data,test_labels)
    print 'Prediction with Multinomial',m_prediction
    
def binarize_data(data, thresholds):
    # Initialize a new feature array with the same shape as the original data.
    binarized_data = np.zeros(data.shape)
    
    #Apply a threshold  to each feature.
    for feature in range(data.shape[1]):
        binarized_data[:,feature] = data[:,feature] > thresholds[0]
    return binarized_data
        
def multinomialize_data(data):
    mult_data = np.zeros(data.shape)
    for point in range(len(data)):
        mult_point = np.zeros(data[point].shape)
        data_point = data[point]
        for feature in range(data_point.size):
            if(data_point[feature] >= 0 and data_point[feature] <= 0.33):
                mult_point[feature] = 0
            elif(data_point[feature] > 0.33  and data_point[feature]<=0.66):
                mult_point[feature] = 1
            else:
                mult_point[feature] = 2
    return mult_data


### STUDENT END ###

P7()

Converting Data to Binomial
Prediction with Binomial 0.838222222222
Converting Data to Multinomial
Prediction with Multinomial 0.115888888889


No The multinomial does not improve the search.

(8) Use GridSearchCV to perform a search over values of alpha (the Laplace smoothing parameter) in a Bernoulli NB model. What is the best value for alpha? What is the accuracy when alpha=0? Is this what you'd expect?

- Note that GridSearchCV partitions the training data so the results will be a bit different than if you used the dev data for evaluation.

In [ ]:
def P8(alphas):

### STUDENT START ###
    b_clf = BernoulliNB()
    GridSearchCV(b_clf, )

### STUDENT END ###

alphas = {'alpha': [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]}
nb = P8(alphas)

In [ ]:
#print nb.best_params_

ANSWER:

(9) Try training a model using GuassianNB, which is intended for real-valued features, and evaluate on the dev data. You'll notice that it doesn't work so well. Try to diagnose the problem. You should be able to find a simple fix that returns the accuracy to around the same rate as BernoulliNB. Explain your solution.

Hint: examine the parameters estimated by the fit() method, theta\_ and sigma\_.

In [66]:
def P9():

### STUDENT END ###
    g_clf = GaussianNB()
    g_model = g_clf.fit(train_data,train_labels)
    print mean(g_clf.sigma_)#g_clf.theta_
    g_predicted = g_model.score(dev_data,dev_labels)
    print g_predicted

### STUDENT END ###

gnb = P9()

NameError: global name 'mean' is not defined

ANSWER:

(10) Because Naive Bayes is a generative model, we can use the trained model to generate digits. Train a BernoulliNB model and then generate a 10x20 grid with 20 examples of each digit. Because you're using a Bernoulli model, each pixel output will be either 0 or 1. How do the generated digits compare to the training digits?

- You can use np.random.rand() to generate random numbers from a uniform distribution
- The estimated probability of each pixel is stored in feature\_log\_prob\_. You'll need to use np.exp() to convert a log probability back to a probability.

In [ ]:
#def P10(num_examples):

### STUDENT START ###


### STUDENT END ###

#P10(20)

ANSWER:

(11) Remember that a strongly calibrated classifier is rougly 90% accurate when the posterior probability of the predicted class is 0.9. A weakly calibrated classifier is more accurate when the posterior is 90% than when it is 80%. A poorly calibrated classifier has no positive correlation between posterior and accuracy.

Train a BernoulliNB model with a reasonable alpha value. For each posterior bucket (think of a bin in a histogram), you want to estimate the classifier's accuracy. So for each prediction, find the bucket the maximum posterior belongs to and update the "correct" and "total" counters.

How would you characterize the calibration for the Naive Bayes model?

In [ ]:
#def P11(buckets, correct, total):
    
### STUDENT START ###


                
### STUDENT END ###

#buckets = [0.5, 0.9, 0.999, 0.99999, 0.9999999, 0.999999999, 0.99999999999, 0.9999999999999, 1.0]
#correct = [0 for i in buckets]
#total = [0 for i in buckets]

#P11(buckets, correct, total)

#for i in range(len(buckets)):
#    accuracy = 0.0
#    if (total[i] > 0): accuracy = correct[i] / total[i]
#    print 'p(pred) <= %.13f    total = %3d    accuracy = %.3f' %(buckets[i], total[i], accuracy)

ANSWER:

(12) EXTRA CREDIT

Try designing extra features to see if you can improve the performance of Naive Bayes on the dev set. Here are a few ideas to get you started:
- Try summing the pixel values in each row and each column.
- Try counting the number of enclosed regions; 8 usually has 2 enclosed regions, 9 usually has 1, and 7 usually has 0.

Make sure you comment your code well!

In [ ]:
#def P12():

### STUDENT START ###


### STUDENT END ###

#P12()